# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
resource="../output_data/city_output"
file=pd.read_csv(resource)
file.head()

,Unnamed: 0,City,Cloudiness,Country,Data,Humanity,Temperature,Latitude,Longitude,Wind Speed
0,0,acarau,100,BR,1580107591,96,297.32,-2.89,-40.12,2.13
1,1,verkhnevilyuysk,100,RU,1580107592,97,245.50,63.45,120.32,1.47
2,2,bredasdorp,90,ZA,1580107753,59,291.15,-34.53,20.04,1.00
3,3,severo-kurilsk,86,RU,1580107780,91,266.23,50.68,156.12,4.48
4,4,sakaiminato,75,JP,1580107592,81,282.15,35.55,133.23,12.90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

humidity=file['Humanity'].astype(float)

city_loc=file[["Latitude","Longitude"]]


In [4]:
heat_map=gmaps.heatmap_layer(city_loc, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig=gmaps.figure(center=(0,0), zoom_level=2)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
new_city=file.loc[(file['Latitude']>=0)&(file['Longitude']>=0)&(file['Temperature']>=30)&(file['Cloudiness']>=50)]
new_city=new_city.dropna(how='all')
len(new_city)


124

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#hotel_df=new_city.reset_index(drop=True)
#len(hotel_df)
hotel_df["Hotel Name"]=""



for x,row in hotel_df.iterrows():
    Lat=row["Latitude"]
    Long=row["Longitude"]
 
    params={"location":f"{Lat},{Long}",
        "radius":5000,
        "types":"hotel",
         "key": g_key}

    
    url_search="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name=requests.get(url_search, params=params)
    hotel_name=hotel_name.json()
    
    try:
        hotel_df.loc[x,"Hotel Name"]=hotel_name['results'][0]['name']
    except:
        hotel_df.append("")
        
hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Data,Humanity,Temperature,Latitude,Longitude,Wind Speed,Hotel Name
0,1,verkhnevilyuysk,100,RU,1580107592,97,245.50,63.45,120.32,1.47,Verkhnevilyuysk
1,3,severo-kurilsk,86,RU,1580107780,91,266.23,50.68,156.12,4.48,Severo-Kurilsk
2,4,sakaiminato,75,JP,1580107592,81,282.15,35.55,133.23,12.90,Matsue
3,9,pevek,100,RU,1580107730,92,253.57,69.70,170.31,1.48,Pevek
4,15,ugoofaaru,100,MV,1580107593,75,300.52,5.67,73.00,4.12,Ungoofaaru


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
heatmap=gmaps.marker_layer(locations, info_box_content=[f"Hotel loc:{rate}" for rate in hotel_info])

fig.add_layer(heatmap)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))